### ספריות


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### העלת משתנים להרצת הקוד


In [ ]:
cwd = os.getcwd()

### פונקציות גלובליות


In [3]:
from global_functions import up_load_df, get_output_folder_location

### הגדרת מיקום הפלט


In [ ]:
# תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
# מיקום תיקיית פלט
output_folder_location=get_output_folder_location(r'{}\background_files\output_folder_location.txt'.format(cwd))
# נתיב שכבת פלט
out_shp = r'{}\education_{}.shp'.format(output_folder_location, file_date)

### יצירת נתוני מוסדות חינוך מעיריות ומשרד החינוך


In [ ]:
# הרצת מחברות
# עיריית ירושלים
%run "jerusalem_muni.ipynb"
# משרד החינוך
%run "ministry_of_education.ipynb"
# עיריית בית שמש
%run "betshemesh_muni.ipynb"

### טעינת נתוני מוסדות חינוך מעיריות ומשרד החינוך


In [ ]:
# בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\Intermediates'.format(cwd),'BShemesh_final')

In [ ]:
# בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\Intermediates'.format(cwd),'JLM_final')

In [ ]:
# בתי ספר ממשרד החינוך במרחב
Gschool=up_load_df(r'{}\Intermediates'.format(cwd),'Gschool_final')

### חיבור הטבלאות


In [9]:
education=pd.concat([BShemesh, JLM, Gschool], ignore_index=True)

### יצירת שכבת חינוך


In [ ]:
# יצירת שכבה גאומטרית
geometry = [Point(xy) for xy in zip(education['x'], education['y'])]
gdf = gpd.GeoDataFrame(education, geometry=geometry, crs='EPSG:2039')

gdf.rename(columns={'SRC': 'coord_src'}, inplace=True)
gdf = gdf.drop(['x', 'y'], axis=1)

### מחיקת מוסדות בלי תלמידים


In [ ]:
gdf = gdf[~((gdf['ele_stu'] == 0) & 
                                  (gdf['mid_stu'] == 0) & 
                                  (gdf['high_stu'] == 0))]

education = education[~((education['ele_stu'] == 0) & 
                                  (education['mid_stu'] == 0) & 
                                  (education['high_stu'] == 0))]


### ייצוא


In [12]:
gdf.to_file(out_shp, driver="ESRI Shapefile")
education.to_excel(r'{}\education_final_{}.xlsx'.format(output_folder_location, file_date), index=False)